#### Mount Elastic NFS 

In [1]:
%%sh

mkdir efs

mkdir: cannot create directory ‘efs’: File exists


In [2]:
%%sh 

sudo mount -t nfs \
    -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 \
    172.31.14.15:/ \
    ./efs

sudo chmod go+rw ./efs

In [1]:
import boto3
import pandas as pd
from sagemaker import get_execution_role
import random
import os
import hashlib
import sys
import json 
import requests

# from utils import logger

In [3]:
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)
# boto3.describe_mount_targets(FileSystemId='<file_system_id>')
pd.__version__

us-east-1


'0.22.0'

##### Generate caseid_fileid matrix (Copy Number Variation & miRNA)

In [22]:
'''
read the json file and parse the file id and case id info and save it 
'''
# CPV file
# json_file = "/efs/manifests/files.2018-11-07-cpv.json"
# outputfile = "file_caseid_cpv.csv"

# miRNA file
# json_file = "/efs/manifests/metadata-miRNA.json"
# outputfile = "file_caseid_miRna.csv"

# DNA Methylation file
# json_file = "efs/manifests/files.2018-11-08-dna_meth.json"
# outputfile = "file_caseid_dnaMeth.csv"

# mRNA file
json_file = "efs/manifests/files.2018-11-08-mrna.json"
outputfile = "file_caseid_mrna.csv"


with open(json_file) as data_file:    
    data = json.load(data_file)
    
data_arr = []
case_ids = set()
for each_record in data:
    # print (each_record)
    file_id = each_record['file_id']
    case_id =  each_record['cases'][0]['case_id']
    if case_id in case_ids:
        case_ids.add(case_id)

    else:

        data_arr.append([file_id,case_id])

df = pd.DataFrame(data_arr, columns = ['file_id','case_id'])

df.to_csv(outputfile,index=False)


##### Request Metadata (Copy Num Variation)

In [94]:
# for CPV
# metafile = "files_meta_cpv.csv"
# for miRNA
# metafile = "files_meta_miRNA.csv"
# for DNAmeth
# metafile = "files_meta_dnaMeth.csv"
# for RNA 
metafile = "files_meta_mRNA.csv"

df = pd.read_csv(outputfile)
case_ids = df.case_id.values
file_ids = df.file_id.values
fd = open(metafile,'w')
size = file_ids.shape[0] + 1000
id_list = file_ids.tolist()
print(len(id_list))
# print(len(id_list[:14713]))

cases_endpt = 'https://api.gdc.cancer.gov/files'
fields = [
    "file_id",
    "file_name",
#     "cases.submitter_id",
    "cases.case_id",
    "data_category",
    "data_type",
    # "cases.samples.tumor_descriptor",
    "cases.samples.sample_type",
    "cases.project.disease_type",
    "cases.diagnoses.primary_diagnosis",
    # "cases.project.name",
#     "cases.project.project_id",
#     "cases.samples.sample_type",
#     "cases.samples.submitter_id",
    "cases.samples.sample_id",
    "cases.samples.portions.analytes.aliquots.aliquot_id",
#     "cases.diagnoses.classification_of_tumor",
#     "cases.samples.portions.slides.percent_tumor_cells",
#     "cases.samples.portions.slides.percent_normal_cells",
#     "cases.samples.portions.slides.percent_stromal_cells"
    # "cases.samples.portions.analytes.aliquots.submitter_id"
    ]


filters = {
    "op":"in",
    "content":{
        "field":"files.file_id",
#         "value": file_ids.tolist()
        "value":id_list[:22200]
    }
}
filters2 = {
    "op":"in",
    "content":{
        "field":"files.file_id",
#         "value": file_ids.tolist()
        "value":id_list[22200:]
    }
}

# print (filters)
fields = ','.join(fields)
#expand group is diagnosis and demoragphic
params = {
    "filters" : filters,
    "fields": fields,
    # "expand" : "diagnoses,demographic,exposures",
    "format": "CSV",
    "pretty": "true",
    "size": size
}

params2 = {
    "filters" : filters2,
    "fields": fields,
    # "expand" : "diagnoses,demographic,exposures",
    "format": "CSV",
    "pretty": "true",
    "size": size
}
# print (params)
#print (filters)
#print (fields)


response = requests.post(cases_endpt, headers = {"Content-Type": "application/json"},json = params)
fd.write(response.content.decode("utf-8"))
response2 = requests.post(cases_endpt, headers = {"Content-Type": "application/json"},json = params2)
fd.write(response2.content.decode("utf-8"))
fd.close()

# edit
meta_RNA = pd.read_csv(metafile)
meta_RNA=meta_RNA.drop(meta_RNA.index[22200])
meta_RNA=meta_RNA.dropna(how='any', axis=1,thresh=1000)
meta_RNA.to_csv(metafile,index=False)

filestat = os.stat(metafile)
print (filestat.st_size)

34713
12586982


In [96]:
# Combine to form file and label matrix
from collections import defaultdict
meta_cpv = pd.read_csv("files_meta_cpv.csv")
meta_miRNA = pd.read_csv("files_meta_miRNA.csv")
meta_dnaMeth = pd.read_csv("files_meta_dnaMeth.csv")
meta_mRNA = pd.read_csv("files_meta_mRNA.csv")


cpv_file_dict = dict(zip(meta_cpv['cases.0.samples.0.sample_id'], meta_cpv['file_id']))
miRNA_file_dict = dict(zip(meta_miRNA['cases.0.samples.0.sample_id'], meta_miRNA['file_id']))
dnaMeth_file_dict = dict(zip(meta_dnaMeth['cases.0.samples.0.sample_id'], meta_dnaMeth['file_id']))
mRNA_file_dict = dict(zip(meta_mRNA['cases.0.samples.0.sample_id'], meta_mRNA['file_id']))

print("# of CPV samples: ",len(cpv_file_dict))
print("# of miRNA samples: ",len(miRNA_file_dict))
print("# of DNA_Meth samples: ",len(dnaMeth_file_dict))
print("# of RNA samples: ",len(RNA_file_dict))

dicts = [cpv_file_dict,
         miRNA_file_dict,
        dnaMeth_file_dict,
        mRNA_file_dict]
super_file_dict = defaultdict(set)  # uses set to avoid duplicates
for d in dicts:
    for k, v in d.items(): 
        super_file_dict[k].add(v)

        
print("# of unique samples:",len(super_file_dict))

# temp = pd.DataFrame.from_dict(super_file_dict, orient='index')
# print(temp.shape)
# temp.columns=["pv","mirna","dna_meth","mrna"]
# temp.index.name="sample_id"
# temp.to_csv("temp.csv",index=True)

# delete entries without both entries 
super_file_dict = { k:v for k,v in super_file_dict.items() if len(v)==len(dicts) }
print("# of unique samples with complete info:",len(super_file_dict))     
# print(dict(list(super_dict.items())[0:3]))
# print(meta_cpv.shape[0])
# print(len(super_dict))


# of CPV samples:  22142
# of miRNA samples:  11422
# of DNA_Meth samples:  12172
# of RNA samples:  11535
# of unique samples: 23166
# of unique samples with complete info: 10133


In [97]:
cpv_label_df = meta_cpv[['cases.0.samples.0.sample_id','cases.0.samples.0.sample_type','cases.0.project.disease_type','cases.0.diagnoses.0.primary_diagnosis','cases.0.case_id']]
cpv_label_dict = cpv_label_df.set_index('cases.0.samples.0.sample_id').T.to_dict('list')
miRNA_label_df = meta_miRNA[['cases.0.samples.0.sample_id','cases.0.samples.0.sample_type','cases.0.project.disease_type','cases.0.diagnoses.0.primary_diagnosis','cases.0.case_id']]
miRNA_label_dict = miRNA_label_df.set_index('cases.0.samples.0.sample_id').T.to_dict('list')
DNA_meth_label_df = meta_dnaMeth[['cases.0.samples.0.sample_id','cases.0.samples.0.sample_type','cases.0.project.disease_type','cases.0.diagnoses.0.primary_diagnosis','cases.0.case_id']]
DNA_meth_label_dict = DNA_meth_label_df.set_index('cases.0.samples.0.sample_id').T.to_dict('list')
mRNA_label_df = meta_mRNA[['cases.0.samples.0.sample_id','cases.0.samples.0.sample_type','cases.0.project.disease_type','cases.0.diagnoses.0.primary_diagnosis','cases.0.case_id']]
mRNA_label_dict = mRNA_label_df.set_index('cases.0.samples.0.sample_id').T.to_dict('list')
# print(miRNA_file_dict['22be9844-4d9e-4372-a46a-b5c64480e5aa'])
# print(cpv_file_dict['22be9844-4d9e-4372-a46a-b5c64480e5aa'])
# print("mRNA entries", len(miRNA_label_dict))
# print("CPV entries",len(cpv_label_dict))


dicts = [cpv_label_dict,
         miRNA_label_dict,
        DNA_meth_dict,
        mRNA_label_dict]
super_meta_dict = defaultdict(set)  # uses set to avoid duplicates
for d in dicts:
    for k, v in d.items(): 
        super_meta_dict[k] = v

# # delete entries without both entries 
for key in list(super_meta_dict):
    if key not in super_file_dict.keys():
        super_meta_dict.pop(key)
        
    
# Check if the same size
# print(len(super_meta_dict))
print(len(super_file_dict))
# print(dict(list(super_meta_dict.items())[0:3]))
# print(dict(list(super_file_dict.items())[0:3]))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.


10133


In [10]:
# Generate combined CPV & miRNA matrix
print(len(super_file_dict))
print(len(super_meta_dict))

cpv_features = pd.read_csv("cpv.csv")
miRna_features = pd.read_csv("miRNA.csv")
df2 = pd.DataFrame.from_dict(super_meta_dict, orient='index')
df2.columns=["sample_type","disease_type","primary_diagnosis","case_id"]
df2.index.name="sample_id"

df = pd.DataFrame.from_dict(super_file_dict, orient='index')
print(df.shape)
df.columns=["file_id_cpv","file_id"]
df.index.name="sample_id"
df = pd.concat([df.reset_index(), miRna_features], axis=1)
df.set_index('sample_id')
df.pop('file_id')
df.pop('disease_type')
df.rename(columns={'file_id_cpv':'file_id'}, inplace=True)
df = pd.concat([df.reset_index(), cpv_features], axis=1)
df.set_index('sample_id')
df.pop('file_id')

# df=df.reset_index().merge(df2,how = 'right',on="sample_id").set_index('sample_id')
df = pd.concat([df.reset_index(), df2.reset_index()], axis=1)
df.set_index('sample_id')

df=df.dropna(axis=0,how='any')

print(df.shape)

df.to_csv("final_matrix.csv",index=False)

10718
10718
(10718, 2)
(10718, 1937)


In [ ]:
%%sh

python efs/cpv/gen_cnv_matrix.py /home/ec2-user/SageMaker/efs/cpv/data files_meta_cpv.csv cpv.csv

#### Download Data from GDC repository
mkdir /tmp/gdcData/ <br>
cd /tmp/gdcData/ <br>
/home/ec2-user/SageMaker/gdc-client download -m /home/ec2-user/SageMaker/manifest.txt<br>

#### Check data integrity 

In [8]:
def file_as_bytes(file):
    with file:
        return file.read()

def check(dirname,manifest_file,total):
    '''
    check the md5 for each file downloaded. if md5 does not match, report error.
    '''
    df = pd.read_csv(manifest_file,sep='\t')
    count = 0
    for idname in os.listdir(dirname):
        # list all the ids 
        if idname.find("-") != -1:
            idpath = dirname +"/" + idname

            for filename in os.listdir(idpath):
                # check the miRNA file
                if filename.find("-") != -1:
                    filepath = idpath + "/" + filename
                    filehash = hashlib.md5(file_as_bytes(open(filepath, 'rb'))).hexdigest()
                    if df.loc[df['filename'] == filename].md5.values[0] != filehash:
                        print("file id {} download fails, please downlaod again".format(idname))
#                         logger.info("file id {} download fails, please downlaod again".format(idname))
                    else:
                        count +=1
    if count == total:
        print("successful downloads")
#         logger.info("successful downloads")

In [ ]:
check("efs/dna_meth/data","efs/dna_meth/gdc_manifest.2018-11-07-dna_meth.txt",12359)

#### Generate DNA Mehtylation matrix

In [ ]:
def extractMatrix(dirname):
    '''
    return a dataframe of the miRNA matrix, each row is the miRNA counts for a file_id

    '''
    count = 0

    miRNA_data = []
    for idname in os.listdir(dirname):
        # list all the ids 
        if idname.find("-") != -1:
            idpath = dirname +"/" + idname

            # all the files in each id directory
            for filename in os.listdir(idpath):
                # check the miRNA file
                if filename.find("-") != -1:
                    filepath = idpath + "/" + filename
                    df = pd.read_csv(filepath,sep="\t")
                    # columns = ["miRNA_ID", "read_count"]
                    if count ==0:
                        # get the miRNA_IDs 
                        miRNA_IDs = df.Beta_value.values.tolist()

                    id_miRNA_read_counts = [idname] + df.read_count.values.tolist()
                    miRNA_data.append(id_miRNA_read_counts)


                    count +=1
                    # print (df)
    columns = ["file_id"] + miRNA_IDs
    df = pd.DataFrame(miRNA_data, columns=columns)
    return df

In [ ]:
matrix_df = extractMatrix("efs/dna_meth/data")
matrix_df.to_csv("da_meth.csv", index=False)

#### Create S3 Bucket

In [ ]:
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)
number = str(random.randint(0,1000))
bucket_name="gdcdataml"+str(number) # <--- change this variable to a unique name for your bucket
s3 = boto3.client('s3')
print("bucketname:",bucket_name)
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

In [ ]:
# cwd = os.getcwd()
# print(cwd)
print(4*"="+"start checking"+4*"=")
manifest_file = cwd+'/gdc_manifest.2018-11-07-all.txt'
df = pd.read_csv(manifest_file,sep='\t')
total = df.shape[0]
check(manifest_file,total)

#### Upload directory

In [ ]:
def uploadDirectory(path,bucketname):
    for root,dirs,files in os.walk(path):
        for file in files:
            s3.upload_file(os.path.join(root,file),bucketname,file)

In [ ]:
raw_data_dir = '/tmp/gdcData/'
uploadDirectory(raw_data_dir,bucket_name)

In [ ]:
# matrix = dict()
# inventory = dict()
# for each_record in data:
#     file_id = each_record['file_id']
#     case_id =  each_record['cases'][0]['case_id']
#     filename = each_record['file_name']
    
#     if case_id in case_ids:
#         case_ids.add(case_id)

#     else:

#         matrix.append([file_id,case_id])
#         inventory[case_id]=[0]*8
    
#     if case_id not in matrix:
#         matrix[case_id]= []
#         inventory[case_id]=[0]*8
    
#     matrix[case_id].append(file_id)
        
#     if ".grch38.seg.v2.txt" in filename: # Copy Num Variation 
#         inventory[case_id][0]+=1
#     elif "nocnv_grch38.seg.v2.txt" in filename: # Masked Copy Num Variation
#         inventory[case_id][1]+=1
#     elif "hg38.txt" in filename: # DNA Methylation 
#         inventory[case_id][2]+=1
#     elif "isoforms.quantification.txt" in filename: # isoform
#         inventory[case_id][3]+=1
#     elif "mirnas.quantification.txt" in filename: # miRNA
#         inventory[case_id][4]+=1
#     elif "FPKM.txt.gz" in filename: # normalized mRNA Gene Expression
#         inventory[case_id][5]+=1
#     elif "FPKM-UQ.txt.gz" in filename: # 75th percentile mRNA Gene Expression
#         inventory[case_id][6]+=1
#     elif "htseq.counts.gz" in filename: # HT-seq mRNA Gene Expression
#         inventory[case_id][7]+=1
        

# df = pd.DataFrame.from_dict(matrix, orient='index').reset_index()
# df2 = pd.DataFrame.from_dict(inventory, orient='index').reset_index()
# df2.columns =['case_id','CNV', 'MCNV', 'DNA-MYL', 'ISO', 'miRNA', 'mRNA_norm', 'mRNA_75','mRNA_HT']
# print(df.shape)
# # filter = (df["CNV"] != "")  & (df["DNA-MYL"] != "") & (df["mRNA_norm"] != "") & (df["miRNA"] != "")
# # dfNew = df[filter]
# # print(dfNew.shape)
# df.to_csv(outputfile,index=False)
# df2.to_csv("inventory.csv",index=False)

In [ ]:
prefix = '/tmp/test'
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, '')).upload_file('train.csv')

In [ ]:
fd = open("files_meta_cpv.csv",'w')
cases_endpt = 'https://api.gdc.cancer.gov/cases'

fields = [
    "files.file_id",
    "files.file_name",
#     "aliquot_ids",
#     "analyte_ids",
#     "case_id",
#     "project.project_id",
    "sample_ids",
#     "files.cases.project.project_id",
#     "files.cases.project.program.name",
#     "primary_site",
#     "disease_type",
#     "diagnoses.vital_status",
#     "demographic.gender",
#     "demographic.race",
#     "demographic.year_of_birth"
#     "exposures.bmi",
    # "exposures.height",
    # "exposures.weight",
    # "exposures.cigarettes_per_day",
    # "exposures.alcohol_history",
    # "exposures.alcohol_intensity",
    # "exposures.years_smoked"
    ]


filters = {
    "op":"in",
    "content":{
        "field":"file_id",
        "value": file_ids[-5:].tolist()
    }
}

# print (filters)
fields = ','.join(fields)
#expand group is diagnosis and demoragphic
params = {
    "filters" : filters,
    "fields": fields,
    # "expand" : "diagnoses,demographic,exposures",
    "format": "CSV",
    "pretty": "true",
    "size": size
}
# print (params)
#print (filters)
#print (fields)


response = requests.post(cases_endpt, headers = {"Content-Type": "application/json"},json = params)
# print (response.content.decode("utf-8"))
fd.write(response.content.decode("utf-8"))
fd.close()